<a href="https://colab.research.google.com/github/vanderbilt-data-science/ai_summer/blob/main/week3_day3_langchain_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Summer Week 3, Day 3
# Langchain Agents

[Agents](https://python.langchain.com/en/latest/modules/agents.html) are a powerful way to allow a LLM to make decisions and access tools in an automated fashion. Agents are ideal when a model needs to access the internet, perform mathematical operations, or access an API or other outside resource to respond to a prompt.

In [45]:
# Download required packages to Colab
!pip install -q langchain
!pip install -q openai
!pip install -q google-search-results
!pip install -q chromadb
!pip install -q tiktoken # needed for chromadb

In [46]:
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchRun
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
import os
from getpass import getpass

In [47]:
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

··········


Let's take 5 minutes to sign up for a SERP API key: https://serper.dev/

In [48]:
serp_api_key = getpass()
os.environ["SERPAPI_API_KEY"] = serp_api_key

··········


## Agent Types

Langchain supports four basic Agent Types, accessing using the AgentType class:

1. ZERO_SHOT_REACT_DESCRIPTION: a basic Agent that chooses tools based on their descriptions. Requires tools, a model, and an input string
2. CONVERSATIONAL_REACT_DESCRIPTION: an Agent that uses memory to keep track of chat history. Can answer questions using tools or previous chats. Requires tools, a model, and a dictionary containing chat history and an input string
3. SELF_ASK_WITH_SEARCH: an Agent that uses a single tool to find factual answers. Requires one tool named "Intermediate Answer", a model, and input.
4. REACT_DOCSTORE: an Agent that uses the [ReAct framework](https://arxiv.org/pdf/2210.03629.pdf) to interact with a document store. Requires a "Search" tool and a "Lookup" tool (must have those exact names, a model, and input. The "Search" tool must search for a document, and the "Lookup" tool must look up a term in the found document.

The easiest to use is the ZERO_SHOT_REACT_DESCRIPTION, since it does not require any specific tools. Simply initialize the agent with the loaded tools (from `load_tools`), model, and `AgentType.ZERO_SHOT_REACT_DESCRIPTION` in the agent parameter.

In [49]:
# Set the LLM
llm = OpenAI(temperature = 0.5)

In [50]:
# Language model already loaded in previous cells
# Now we add tools to the llm

tools = load_tools(["serpapi", "llm-math"], llm = llm)

In [51]:
# initialize agent with tools, llm, and agent type
agent = initialize_agent(tools, llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

In [52]:
agent.run("What was the high temperature in Nashville yesterday in Fahrenheit? What is that number raised to the 0.23 power?")



> Entering new AgentExecutor chain...
 Temperature is a current event, so I should search for it.
Action: Search
Action Input: "high temperature Nashville yesterday Fahrenheit"
Observation: High: 69.8ºf @12:00 AM Low: 59ºf @6:15 AM Approx. Precipitation / Rain Total: in.
Thought: I should use the calculator to raise the temperature to the 0.23 power
Action: Calculator
Action Input: 69.8^0.23
Observation: Answer: 2.6551358533283302
Thought: I now know the final answer
Final Answer: 2.6551358533283302

> Finished chain.


'2.6551358533283302'

### Breakout Exercise (10 mins)

Using the agent we created, come up with a new question to ask the model. What decisions did the agent make? Did it arrive at the right answer? Be prepared to share interesting results!

If you have time, play around with the tools you give the agent by adding new tools or taking away previous tools. How do its answers change? 

## Conversation Agent

The Conversation Agent uses memory to act more like a chat bot by taking message history into account. To set up a Conversation Agent, you first need to set up its memory. For this example, we will use the ConversationBufferMemory and give it the name "chat_history". 

In [53]:
memory = ConversationBufferMemory(memory_key="chat_history")

Then, we can pass our memory into the memory parameter for the `initialize_agent` function. 

In [54]:
agent = initialize_agent(tools, llm, agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose = True, memory = memory)


Let's try out our bot.

In [55]:
agent.run(input ="What was the high temperature in Nashville yesterday in Fahrenheit?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Search
Action Input: High temperature in Nashville yesterday in Fahrenheit
Observation: High: 69.8ºf @12:05 AM Low: 59ºf @5:10 AM Approx. Precipitation / Rain Total: in.
Thought: Do I need to use a tool? No
AI: The high temperature in Nashville yesterday in Fahrenheit was 69.8ºf.

> Finished chain.


'The high temperature in Nashville yesterday in Fahrenheit was 69.8ºf.'

In [56]:
agent.run(input = "What is that number raised to the 0.23 power?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 69.8^0.23
Observation: Answer: 2.6551358533283302
Thought: Do I need to use a tool? No
AI: The answer is 2.6551358533283302.

> Finished chain.


'The answer is 2.6551358533283302.'

Notice that this Agent first asks itself if it needs a tool to answer your query. 

### Breakout Exercise (10 mins)
Play around with the ConversationAgent. How does its behavior differ from the ZeroShotReactDescription Agent?

## VectorStore Agent

Let's create our example VectorStores. The first VectorStore is the state of the union text file, which we grab using `wget`. The next VectorStore is the FAQ documentation for the RUFF library, which we grab from the web and load using Langchain's WebBaseLoader.

In [57]:
!wget https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt

--2023-05-25 19:43:19--  https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39027 (38K) [text/plain]
Saving to: ‘state_of_the_union.txt’

state_of_the_union. 100%[===================>]  38.11K  --.-KB/s    in 0.003s  

2023-05-25 19:43:19 (12.4 MB/s) - ‘state_of_the_union.txt’ saved [39027/39027]



In [58]:
from langchain.document_loaders import TextLoader

# Assign Loader to document
loader = TextLoader('./state_of_the_union.txt')

# Load documents
documents = loader.load()

# Create text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Split texts
texts = text_splitter.split_documents(documents)

# Create embeddings 
embeddings = OpenAIEmbeddings()

# Apply embeddings to texts, and upload those to a Chroma database
state_of_union_store = Chroma.from_documents(texts, embeddings, collection_name="state-of-union") # Note we can assign it a collection name 

Agents only interact with tools and input. To allow your Agent to access a database, you must initialize the database as a tool. We can accomplish this by using Langchain's VectorStoreToolkit, a class that creates a tool from a VectorStore, given the VectorStore and it's metadata (name, description, etc.).

In [59]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

# Create the VectorStoreInfo (so our Agent understands what information our VectorStore holds)
vectorstore_info = VectorStoreInfo(
    name="state_of_union_address",
    description="the most recent state of the Union adress",
    vectorstore=state_of_union_store
)

# Give the VectorStoreInfo to our VectorStoreToolkit to create the tool
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)

# Use create_vectorstore_agent to initalize an Agent specifically for searching databases
agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [60]:
agent_executor.run("What did biden say about ketanji brown jackson is the state of the union address?")



> Entering new AgentExecutor chain...
 I need to find the answer in the state of the union address
Action: state_of_union_address
Action Input: What did biden say about ketanji brown jackson
Observation:  Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer
Final Answer: Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.

> Finished chain.


"Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."

### Using Multiple VectorStores

Sometimes your information may be spread across multiple VectorStores. In that case, your Agent will need to decide when to use which VectorStore. This is called "routing," and requires the VectorStoreRouterToolkit and the `create_vectorstore_router_agent`.

In [61]:
from langchain.document_loaders import WebBaseLoader

# Assign loader to document (in this case, a website)
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

# Load documents
docs = loader.load()

# Split text (using text splitter from previous cell)
ruff_texts = text_splitter.split_documents(docs)

# Apply embeddings (using embeddings from previous cell) and upload to Chroma database
ruff_store = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")  # Note we can assign it a collection name

In [62]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_router_agent,
    VectorStoreRouterToolkit,
    VectorStoreInfo,
)

In [63]:
# Create VectorStoreInfo (note we already created vectorstore_info for state of the union address)

ruff_vectorstore_info = VectorStoreInfo(
    name="ruff",
    description="Information about the Ruff python linting library",
    vectorstore=ruff_store
)

# Give ruff_vectorstore_info and vectorstore_info to VectorStoreRouterToolkit
router_toolkit = VectorStoreRouterToolkit(
    vectorstores=[vectorstore_info, ruff_vectorstore_info],
    llm=llm
)

# Use create_vectorstore_router_agent to inialize an Agent for routing between databases
agent_executor = create_vectorstore_router_agent(
    llm=llm,
    toolkit=router_toolkit,
    verbose=True
)

In [64]:
agent_executor.run("What did biden say about ketanji brown jackson is the state of the union address?")



> Entering new AgentExecutor chain...
 I should use the state of the union address tool to answer this question.
Action: state_of_union_address
Action Input: What did biden say about ketanji brown jackson
Observation:  Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer
Final Answer: Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.

> Finished chain.


"Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."

In [65]:
agent_executor.run("What tool does ruff use to run over Jupyter Notebooks?")



> Entering new AgentExecutor chain...
 I should look for a tool that is specific to ruff
Action: ruff
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks.
Thought: I now know the final answer
Final Answer: Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks.

> Finished chain.


'Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks.'

In [66]:
agent_executor.run("What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?")



> Entering new AgentExecutor chain...
 Ruff is a python linting library, and the state of the union address is a speech.
Action: ruff
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks.
Thought: The state of the union address is a speech, so it is unlikely that the president would mention a specific tool.
Final Answer: Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks. It is unlikely that the president mentioned this tool in the state of the union address.

> Finished chain.


'Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks. It is unlikely that the president mentioned this tool in the state of the union address.'

### Class Discussion
What would you use Agents for (personally or for work/research/projects)? 



What would you **not** use Agents for?

 Do any interesting applications come to mind?

## Problems with Agents

What are some problems you have noticed with Agents? Could these be solved somehow?

Something to consider - https://github.com/hwchase17/langchain/issues/4102

## Homework

Prepare a solution to your own problem using Agents. Assess how the Agent did. What type of Agent did you use? 

Play around with the temperature of the model. How does that affect the results?

In [ ]:
# YOUR CODE HERE #